In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk, re, string, collections
from nltk.util import ngrams
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import gensim
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


In [25]:
import plotly.offline as pyoff
import plotly.graph_objs as go

In [26]:
import plotly.graph_objs as go

In [31]:
fake = pd.read_csv('./Fake.csv')
real = pd.read_csv('./Real.csv')

In [32]:
fake['Target']=1
real['Target']=0

In [33]:
frames = [fake, real]

df = pd.concat(frames)

In [34]:
len(df)

44898

In [35]:
df = df.sample(frac=1).reset_index(drop=True)

In [45]:
df_=df.copy()
df_=df_.sort_values(by=['date'])
df_=df_.reset_index(drop=True)


In [46]:
df_['news']=df_['subject']+' '+df_['title']+' '+df_['text']
df_['news'] = df_.apply(lambda x: x['news'].lower(),axis=1)
df_["news"] = df_['news'].str.replace('[^\w\s]','')
all_news=pd.DataFrame(pd.Series(' '.join(df_['news']).split()).value_counts())
allnews1=all_news.head(30)


In [54]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [48]:
def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english')
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [55]:
words = basic_clean(''.join(str(df_['news'].tolist())))


# **Sentence Embedding**

In [120]:
Y = df_['Target'].values

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [99]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [122]:
train, test = train_test_split(df_, test_size=0.3, random_state=42)

In [123]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['news']), tags=[r.Target]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['news']), tags=[r.Target]), axis=1)

In [124]:
train_tagged.values[30]


TaggedDocument(words=['news', 'the', 'united', 'states', 'government', 'has', 'told', 'trump', 'to', 'go', 'fck', 'himself', 'when', 'trump', 'began', 'his', 'disinformation', 'campaign', 'and', 'then', 'started', 'trying', 'to', 'muzzle', 'federal', 'agencies', 'employees', 'of', 'some', 'of', 'these', 'agencies', 'went', 'to', 'twitter', 'to', 'openly', 'defy', 'him', 'they', 'call', 'themselves', 'altgov', 'and', 'there', 'are', 'dozens', 'of', 'these', 'accounts', 'that', 'are', 'making', 'point', 'of', 'pushing', 'out', 'the', 'very', 'facts', 'that', 'trump', 'wishes', 'would', 'disappear', 'as', 'well', 'as', 'openly', 'fighting', 'his', 'alternative', 'facts', 'and', 'overall', 'disinformation', 'campaign', 'many', 'have', 'been', 'verified', 'by', 'snopes', 'who', 'now', 'keeps', 'running', 'list', 'of', 'trusted', 'altgov', 'accountshowever', 'no', 'matter', 'who', 'they', 'are', 'they', 'are', 'all', 'very', 'clear', 'about', 'the', 'fact', 'that', 'they', 'are', 'not', 'doi

In [125]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [129]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)


In [ ]:
model_dbow.build_vocab([x for x in train_tagged.values])

In [139]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [140]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)


In [141]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [142]:
from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.502746844840386
Testing F1 score: 0.4857423751572272


# **Word Emebedding**

In [145]:
y = df_['Target'].values
X = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df_["news"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    X.append(tmp)

In [146]:
EMBEDDING_DIM = 100

w2v_model = gensim.models.Word2Vec(sentences=X, size=EMBEDDING_DIM, window=5, min_count=1)

In [147]:
train, test = train_test_split(df_, test_size=0.3, random_state=42)